<a href="https://colab.research.google.com/github/MKCA06/SDE_Sheet_Striver/blob/main/accident_entity_recon_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk
from nltk.tokenize import word_tokenize
from nltk import ne_chunk
import re
import spacy
from spacy import displacy
import en_core_web_lg
from dateutil import parser
import dateparser
from spacy.pipeline import EntityRuler ####IMPORTING LIBRARIES

In [ ]:
safety=pd.read_excel('D:/Arpit/G drive/FY 22 23 Road Accident.xlsx')

RENAMING COLUMN NAMES

In [ ]:
safety.rename(columns = {'Unnamed: 0':'S.No', 'Unnamed: 1':'DIV',
                              'Unnamed: 2':'Deptt','Unnamed: 3':'Employee type','Unnamed: 4':'Date','Unnamed: 5':'Injury','Unnamed: 6':'Fracture','Unnamed: 7':'Details','Unnamed: 8':'Category','Unnamed: 9':'Direct Cause'}, inplace = True)
safety.drop(index=0,inplace=True)
safety.reset_index(inplace=True)
safety.drop(columns=['index'],inplace=True)

LOCATION DICTIONARY

In [ ]:
loca='''ajmeri gate extension
a g c r
ansari road
aliganj
anand parbat
anand nagar
army base
a p s colony
a f palam
ashram
ashok nagar
ashoka hotel
anand niketan
amar colony
adarsh nagar
a t mills
anandvas shakurpur
alipur
a f rajokari
ambrohi
arjan garh
asian games village
andrews ganj
azad nagar
anarkali
ashok vihar h o
amrit kaur market
ambedkar nagar
bank street
bara tooti
birla lines
bazar road
bhola nath nagar
balbir nagar
bhaiparma nand nagar
bawana
budh nagar
badli
badarpur t p station
badarpur
bakrola
bharat nagar
bhajanpuri
brahampuri
bijwasan
b s f camp tigri
bhajan pura
connaught place
constitution house
chawri bazar
chandni chowk
c c institute
central ordinance
central vehicle
central tractor
central kirti nagar
chand nagar
c r r i
chankyapuri
civil lines
c r p f camp
chhawla
chilla villege
darya ganj
delhi high court
desh bandhu gupta road
delhi sadar bazar
delhi g p o
delhi cloth mills
dareeba
delhi university
dada ghosh bhawan
depot
dhaula khuan
delhi cantt
defence head qrs
dargah shareef
d i area
defence colony
dilshad garden
distt courts
dakshinpuri phase iii
dakshimpuri phase i
d d a munirka
d d a salam
eastern court
election commission
e s i hospital
east of kailash
farashkhana
fatehpuri
fateh nagar
flatted factories complex
gandhi samarak
golf links
guru gobind singh marg
gymkhana club
green park market
green park
govindpuri
gandhi nagar bazar
gandhi nagar
geeta colony
gujranwala nagar part ii
g t s nagar
ganeshpuri
gurgaon road
greater kailash ii
greater kailash
guatam nagar
gulmohar park
gobindpura
gokulpuri
hamdard dawakhana
hauz qazi
hazrat nizamuddin
hari nagar ashram
hauz khas market
hauz khas
himayunpur extn
harijan sewak sangh
haiderpur
hamdard nagar
hari nagar da & db block
hari nagar be block
himmatpuri
indian agriculture
indraprastha estate
indraprastha h o
industrial area
idgah road i s b t
inderpuri
industrial area
i i t hauz khas
inderlok
janpath
jawahar lal nehru stadium
joshi nagar
jama masjid
jawahar nagar
jangpura
jamia nagar
janta market
j 6 block rajouri garden
j j colony
jahangirpuri block h
jahangirpuri block d
jahangirpuri block a
jaroda majra
jawalpuri
jagjit nagar
janakpuri block c 4
janakpuri
janakpuri block d 1
janakpuri blockc 2
jeewan park
jawahar lal nehru univers
jharoda kalan
krishi bhawan
karol bagh
kamla nagar
kirby place
karampura
kirti nagar
khyalla phase ii
khyalla phase i
kalkaji
kidwai nagar
kidwai nagar west
krishna market
kailash nagar
kabool nagar
kailash colony
kailash
krishan nagar bazar
krishan nagar h o
khanpur
kanjhawala
khera kalan
kutubgarh
khichripur
kalyanpuri
kalyan vas
karawal nagar
lady harding medical college
laxminarain mandir
lok nagar bhawan
lodi road h o
lodi house hostel
lodi colony market
lower camp anand parbat
lancer road market
lok nayak bhawan
laxmi bai nagar
lajpat nagar
loni road
lawrence road
lekhu nagar
lajpat rai chowk
laxmi nagar market
minto road
model basti
mori gate
malka ganj market
malka ganj
area name
mansarover garden
moti nagar
malviya nagar
mahabir nagar
major bhupinder singh nag
malcha marg
moti bagh
mehrau;i
mansarover park
mukerjee nagar
model town a block
model town ii
multani mohalla
masjidmoth phase ii
masjid moth extn
maujpur
maidens hotel
multani dhanda
munirka
madangir
madipur
mayapuri shopping centre
mayapuri
mangolpuri block s
mangolpuri block a
mangolpuri block 1
mangolpuri
north avenue
new delhi g p o
nidhi
nirman bhawan
n p laboratories
north industrial area
n i e compus
nehru place
nanakpura
netaji nagar
najafgarh road
naraina vihar
naraina
naraina industrial estate
nauroji nagar
new seemapuri
new sabzi mandi
nirankari colony
nangli poona
narela town
narela
nagloi
nangloi phase iii
najafgarh
nangal rava
n block greater
nimri
new rajinder nagar
nehru nagar
nand nagri a block
nand nagri
nathu colony
oberei intercontinental
organisation
okhla industrial estate
onkar nagar
parliament house
patiala house
parliament street h o
pragati maidan
pandara road
pragati vihar
patrachar road
patrachar vidyala
partap nagar
pragati vihar
patel nagar east
patel nagar
patel nagar west
patel nagar south
pushpa vihar sector i
panchseel enclave
postal saving bureau
punjabi bagh sector iii
punjabi bagh
pitampura
power house
palam enclave
palam
paharganj
p s b paharganj
possangipur
pushpa bhawan
paschim vihar
paschim vihar block b
p t s jharoda kalan
patper ganj
qr madipur
rail bhawan
rajghat power house
rashtrapati bhawan
red fort
rly c a o
rana pratap bagh
roshan area road
roop nagar
research institute
ramesh nagar h o
r k puram sector i
r k puram sector ii
r k puram sector iii
r k puram sector iv
r k puram sector xii
r k puram sector ix
r k puram v
r k puram sector vii
r k puram sector viii
rajouri market
rajouri garden
raghubarpura
rohtash nagar
rani bagh
radheypuri
ram nagar
ravinder rangsala
rajinder nagar
r k puram west
r k puram
shastri bhawan
supreme court
sectt north
sansadiya soudha
safdarjang air port
sat nahar
sadar thana road
shakti nagar
subzi mandi
sewa sangh
safdarjang air port
south avenue
sectt south
shahjahan road
sudershan park
shri aurobindo
south malviya nagar
sarojini nagar h o
subhash nagar
subhash nagar west
safdarjang enclave
sahstri nagar
seemapuri old
shahdara mandi
shahdara
suraj nagar
shalimar bagh
shakurpur block i
srinagar colony
saraswati vihar
shakurbasti r s
sarai rohila
sanoth
sultanpuri block f
suttanpuri block b
sultanpuri block c
sameypur badli
sadiq nagar
south extn part ii
south extn part i
shastri nagar
satyawati nagar
seelampur
swami ram tirath nagar
shakurbasti
sriniwaspuri
shakarpur
than singh nagar
timarpur
tilak nagar
tihar
tilak nagar east
tatarpur
t b hospital
teliwara
tughlakabad
tughlakabad a f station
trilokpuri
udyog bhawan
uttam nagar
ujwa
vishnu garden
vikaspuri
vishnu garden
vishwa karma nagar
vishwan nagar
vivek vihar
vijay nagar
vasant vihar i
vasant vihar ii
wazir nagar
wazir nagar
work shop
wazirpur phase iii
yusaf sarai
yamuna vihar
zakhira
zakhir nagar
zafrabad'''
locat = loca.split('\n')
location_m=locat+['deeghot village','baliyana village','chintapuri mandir','wajirpur','gaon chowk','krishna chawk','mata road','bulandshahar','agra','mathura','bolni','bawal','budhera','rohtak','dhigal','dundahera','farrukhnagar', 'garhi harsaru','gurgaon rural','haileymandi','manesar','pataudi', 'sohna, sukhrali','taoru','dharuhera']


In [ ]:
nlp = en_core_web_lg.load()
ruler=nlp.add_pipe('entity_ruler')

patterns=[{"label":"GPE","pattern":location} for location in location_m]
ruler.add_patterns(patterns)
def extract_transport_mode(text):
    transport_modes=['two wheeler','2 wheeler','four wheeler','2 wheeler','2 w','4 w','three wheeler ','3 wheeler','3 w','auto rickshaw','auto-rickshaw','scooter','tempo','tampo','auto','motorcycle','car','bike','bus','train','walk','train','cycle','bicycle','scooty','walking']
    tokens=nltk.word_tokenize(text.lower()) #pedestrian
    ab=[]
    for token in tokens:
        if token in transport_modes:
            ab.append(token)
    ab=list(set(ab))
    return ab
def extract_direct_cause(text):  ##### collision, hit, slip, imbalance, obstacle,
    cause=['collide','fell','wrong','right','left','disbalanced','lost','slip','collision','hit','slipped','imbalance','obstacle','wrong','misbalance','skid','hit','accident','dog','cattle','cat','cow','animal','collided']
    tokens=nltk.word_tokenize(text.lower()) #pedestrian
    ab=[]
    for token in tokens:
        if token in cause:
            ab.append(token)
    ab=list(set(ab))
    return ab


# def extract_location(text):
#     location_m=locat+['bans gaon chowk','krishna chawk','mata road','bulandshahar','agra','mathura','bolni','bawal','budhera','rohtak','dhigal','dundahera','farrukhnagar', 'garhi harsaru','gurgaon rural','haileymandi','manesar','pataudi', 'sohna, sukhrali','taoru','dharuhera']
#     tokens2=nltk.word_tokenize(text.lower())
#     for token1 in tokens2:
#         if token1 in location_m:
#             return token1
def extract_location2(text):

    doc=nlp(text.lower())
    locations=[ent.text for ent in doc.ents if ent.label_== "GPE"]
    return locations

def extract_shift(text):
    shift_patterns={
        'A shift': r'A shift',
        'A-shift': r'A-shift',
        'B shift': r'B shift',
        'B-shift': r'B-shift',
        'C shift': r'C shift',
        'C-shift': r'C-shift',
        'G-shift': r'G-shift',
        'G shift': r'G shift',
    }
    shift_times=[]
    for shift, pattern in shift_patterns.items():
        matches=re.findall(pattern, text, re.IGNORECASE)
        if matches:
            shift_times.append(shift)
    return shift_times
def extract_time(text):
    doc=nlp(text)
    time=[]
    for X in doc.ents:
        if X.label_=='TIME':
            time.append(X.text)

    return time
def object_hinderance(text):
     hind=['hit','accident','dog','cattle','cat','cow','animal','collided']
     tokens2=nltk.word_tokenize(text.lower())
     for token1 in tokens2:
         if token1 in hind:
             return 1
     return 0
def Person_fault(text):
     if object_hinderance(text)==0:
         fault=['fell','wrong','right','left','disbalanced','lost','slip']
         tokens2=nltk.word_tokenize(text.lower())
         for token1 in tokens2:
             if token1 in fault:
                 return 1
         return 0
     return 0
def check_common_element(list1,list2):
    for element in list1:
        if element in list2:
            return 1
    return-1
def severity_safe(text,text2):
     main=['arms','legs','injuries','toe','forehead','swelling','brushes','neck','elbow','injury','back','thigh','knee','ankle','feet','foot','pain','leg','arm','finger','thumb','hand','wrist','head','face','nose','ears','chin','fatal','shoulder']
     safe=['arms','legs','injuries','toe','swelling','brushes','elbow','injury','back','thigh','knee','ankle','feet','foot','pain','leg','arm','finger','thumb','hand','wrist']
     mild=['forehead','neck','facial','head','face','nose','ears','chin','shoulder']
     severe=['forehead','facial','head','face','nose','ears','chin','shoulder','fatal']
     tokens=nltk.word_tokenize(text.lower())
     ab=[]
     for token in tokens:
         if token in main:
             ab.append(token)
     ab=list(set(ab))
     if text2.lower()=='no' and check_common_element(ab,safe)==1 and check_common_element(ab,mild)==-1 and check_common_element(ab,severe)==-1:
         return 'Minor'
     if (text2.lower()=='yes' and check_common_element(ab,safe)==1 and check_common_element(ab,mild)==-1 and check_common_element(ab,severe)==-1) or (text2=='No' and check_common_element(ab,mild)==1):
         return 'Severe'
     if (text2.lower()=='yes' and  check_common_element(ab,mild)==1 and check_common_element(ab,severe)==1) or (text.lower()=='fatal') or (text2.lower()=='fatal'):
         return 'Major'


c:\Users\INTERN_ASharma\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
entity=[]
transport=[]
loc=[]
shifts=[]
time=[]
ob=[]
per=[]
seve=[]
caus=[]
for i in range(len(safety)):
    transport_mode=extract_transport_mode(safety['Details'][i])
    location=extract_location2(safety['Details'][i])
    shif=extract_shift(safety['Details'][i])
    tim=extract_time(safety['Details'][i])
    obj_hind=object_hinderance(safety['Details'][i])
    pers_faul=Person_fault(safety['Details'][i])
    sev=severity_safe(str(safety['Injury'][i]),str(safety['Fracture'][i]))
    ca=extract_direct_cause(safety['Details'][i])

    loc.append(location)
    transport.append(transport_mode)
    shifts.append(shif)
    time.append(tim)
    ob.append(obj_hind)
    per.append(pers_faul)
    seve.append(sev)
    caus.append(ca)



In [ ]:
safety2=safety.copy()

In [ ]:
safety2['Transport']=transport
safety2['location']=loc
safety2['shifts']=shifts
safety2['Time']=time
safety2['Object Hinderance']=ob
safety2['person fault']=per
safety2['Accident Severity']=seve
safety2['Direct Cause']=caus

In [ ]:
safety2

,Outside Road Accident Details,DIV,Deptt,Employee type,Date,Injury,Fracture,Details,Category,Direct Cause,Transport,location,shifts,Time,Object Hinderance,person fault,Accident Severity
0,1,PACC,SND-M,CW,2021-04-03 00:00:00,Ramnivas - Abrasion on Right Leg \nKrishna - P...,NO,While coming to company in A shift at around 6...,NaN,"[hit, wrong]","[bike, scooty]",[gaon chowk],[A shift],[around 6:00 AM],1,0,Minor
1,2,VEH-M,MPMA,AM,2021-04-04 00:00:00,Internal Pain,NO,"While moving towards right side for sector 6,...",NaN,"[hit, right]",[car],[dharuhera],[],[around 4:30 PM],1,0,Minor
2,3,MPT,RR facilities MPT,Contractor,2021-04-04 00:00:00,Right hand fracture,Yes,"While coming from Dhigal (Rohtak District), th...",NaN,[],[tempo],"[dhigal, rohtak]",[],[],0,0,Severe
3,4,VEH-G,WS2-3,Apprentice,2021-04-07 00:00:00,"Head injury, Fracture in Lag and hand",Yes,"Around 1:00 AM, while going back to home by bi...",NaN,[hit],"[bus, bike]",[budhera],[],[Around 1:00 AM],1,0,Major
4,5,MPT,TCMTW,SE,2021-04-10 00:00:00,Right Leg Fracture & Right hand injured,Yes,Bike hit with electricity pole at Bolni near b...,NaN,[hit],"[bus, bike]",[bolni],[C shift],[],1,0,Severe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,220,VEH-M,PS-C,Regular,2023-03-06 00:00:00,_,_,# B shift departure Buses of R no 16A(Jhajjar)...,NaN,"[hit, accident, wrong]",[bus],[jhajjar],[B shift],[],1,0,None
219,221,VEH-G,WS1,CW,2023-03-06 00:00:00,Injuries on shoulder and LH lower leg,Yes,# Amit was returning back to his home after co...,NaN,"[hit, fell]",[bike],[],[B shift],[],1,0,Major
220,222,VEH-M,PS-MC,CW,2023-03-07 00:00:00,NaN,NaN,"In B shift, due to traffic jam company bus sto...",NaN,"[hit, wrong]",[bus],[],[B shift],[],1,0,None
221,223,VEH-G,AS-3,CW,2023-03-18 00:00:00,NaN,NaN,"While coming to duty from bike, his bike got s...",NaN,"[right, fell]",[bike],[],[],[],0,1,None


In [ ]:
#safety2.to_csv('D:/Arpit/G drive/safety_split8.csv')